# Named Entity Recognition with Pytorch and 🔭 Galileo

In this tutorial, we'll train a model with PyTorch and explore the results in Galileo.

**Make sure to select GPU in your Runtime! (Runtime -> Change Runtime type)**

In [ ]:
#@title Install `dataquality`

# A small function for minimizing the dataset for testing
import os

def _minimize_for_ci() -> bool:
    return os.getenv("MINIMIZE_FOR_CI", "false") == "true"

try:
    import dataquality as dq
except ImportError:
    # Upgrade pip
    !pip install -U pip &> /dev/null

    # Install all dependecies
    !pip install -U dataquality torch datasets transformers &> /dev/null
    
    print('👋 Installed necessary libraries and restarting runtime! This should only need to happen once.')
    print('🙏 Continue with the rest of the notebook or hit "Run All" again!')

    # Restart the runtime
    import os, time
    time.sleep(1) # gives the print statements time to flush
    os._exit(0) # exits without allowing the next cell to run

In [ ]:
import dataquality as dq
dq.__version__

# 1. Login to Galileo

In [ ]:
import dataquality as dq

dq.login()

# 2. Load Data

In [ ]:
#@title 🤗 HuggingFace Dataset
#@markdown You can select any dataset from [here](https://huggingface.co/datasets?language=language:en&task_categories=task_categories:token-classification&task_ids=task_ids:named-entity-recognition&sort=downloads) which contains train/test splits and an `ner_tags` column.

dataset_name = 'conllpp' #@param ["wnut_17", "conllpp", "wikiann"] {allow-input: true}
print(f"You selected the {dataset_name} dataset")

from IPython.utils import io
from datasets import load_dataset, get_dataset_config_names

# Try to load the data. If a config (subset) is needed, pick one
try:
  with io.capture_output() as captured:
    data = load_dataset(dataset_name)
except ValueError as e:
  if "Config name is missing" not in repr(e):
    raise e

  configs = get_dataset_config_names(dataset_name)
  print(f"The dataset {dataset_name} has multiple subsets {configs}.")
  config = input(f"🖖 Enter the name of the subset to pick (or leave blank for any): ")
  if config:
    assert config in configs, f"{config} is not a valid subset"
  else:
    config = configs[0]
  with io.capture_output() as captured:
    data = load_dataset(dataset_name, name=config)

if _minimize_for_ci():
  data["train"] = data["train"].select(range(10))
  data["test"] = data["test"].select(range(10))

# Check that the dataset has at least train and test splits
assert {"train", "test"}.issubset(data), \
f"💾 The dataset {dataset_name} does no have train/test splits, please pick another one."

print(f"\n🏆 Dataset {dataset_name} loaded succesfully")

# 3. Initialize Galileo

In [ ]:
# 🔭🌕 Initializing a new run in Galileo. Each run goes under a project.
dq.init(task_type="text_ner", 
        project_name="named_entity_recognition_pytorch", 
        run_name=f"example_run_{dataset_name.replace('/','-')}")

# 4. Tokenize and Log Dataset
Galileo provides HF integrations to allow tokenization, and label alignment, while automatically logging your input data. Additionally, you can also use the `get_dataloader` function instead of PyTorch's native `DataLoader` to autoatically create a `Dataset` class, and prepare a dataset loader for each split.  

In [ ]:
import pandas as pd
from transformers import DataCollatorForTokenClassification
from dataquality.integrations import hf
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer)

MINIBATCH_SIZE = 32
NUM_EPOCHS = 5

# 🔭🌕 Galileo tokenizes the HuggingFace DatasetDict logs the dataset(s) present in it
tokenized_datasets = hf.tokenize_and_log_dataset(data, tokenizer)
labels = tokenized_datasets['train'].features['ner_tags'].feature.names  

train_dataloader = hf.get_dataloader(tokenized_datasets["train"], collate_fn=data_collator, batch_size=MINIBATCH_SIZE, shuffle=True)
test_dataloader = hf.get_dataloader(tokenized_datasets["test"], collate_fn=data_collator, batch_size=MINIBATCH_SIZE, shuffle=False)

# 5. Prepare and Log NER Model 

One line of Galileo code to log the NER Model data (via `log_model_outputs`). This step logs the model logits and embeddings. 

In [ ]:
import torch

class TokenClassificationModel(torch.nn.Module):
  """Defines a Pytorch text classification bert based model."""

  def __init__(self, num_labels: int):
    super().__init__()
    self.feature_extractor = AutoModel.from_pretrained("distilbert-base-uncased")
    self.classifier = Linear(self.feature_extractor.config.hidden_size, num_labels)

  def forward(self, ids, x, attention_mask):
    """Model forward function."""
    encoded_layers = self.feature_extractor(
        input_ids=x, attention_mask=attention_mask
    ).last_hidden_state
    logits = self.classifier(encoded_layers)

    # 🔭🌕 Galileo logging
    dq.log_model_outputs(
        embs=encoded_layers[:, 1:, :], logits=logits[:, 1:, :], ids=ids
    )
    return logits

# 6. Putting into Action: Training a Model

Complete the training pipeline using a standard PyTorch training setup. While training, Galileo logs the current `epoch` and `split`. 

Call `dq.finish()` after training to complete the logging.

In [ ]:
from torch.nn import CrossEntropyLoss
import torch
import torch.nn.functional as F
from torch.nn import Linear
from transformers import AutoModel
from tqdm.notebook import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = TokenClassificationModel(len(labels))
model.to(device)
print(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)
loss_fn = CrossEntropyLoss(ignore_index=-100)

for epoch in tqdm(range(NUM_EPOCHS)):
    # 🔭🌕 Galileo logging
  dq.set_epoch(epoch)

  model.train()
  train_loss = 0.0
  num_train_batches = 0
  for train_minibatch in tqdm(train_dataloader):
    optimizer.zero_grad()
    # 🔭🌕 Galileo logging
    dq.set_split("train")
    logits = model(train_minibatch["id"], train_minibatch["input_ids"].to(device),
                    train_minibatch["attention_mask"].to(device))
    loss = loss_fn(logits.transpose(1, 2), torch.tensor(train_minibatch["labels"]).to(device))
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    num_train_batches += 1
    if num_train_batches % 50 == 0:
      print('Loss', loss.item(), 'num_batch', num_train_batches)

  with torch.no_grad(): 
    test_loss, num_test_batches = 0.0, 0
    for test_minibatch in tqdm(test_dataloader):
      # 🔭🌕 Galileo logging
      dq.set_split("test")
      logits = model(test_minibatch["id"], test_minibatch["input_ids"].to(device), test_minibatch["attention_mask"].to(device))
      loss = loss_fn(logits.transpose(1, 2), torch.tensor(test_minibatch["labels"].to(device)))
      test_loss += loss.item()
      num_test_batches += 1

    print(f"Finished with epoch {epoch}. Test Loss {test_loss / num_test_batches}")

# 🔭🌕 Galileo logging
dq.finish()

# General Help and Docs
- To get help with your task's requirements, call `dq.get_data_logger().doc()`
- To see more general data and model logging docs, run `dq.docs()`

In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)